# Uncertainty Analysis
This tutorial shows how to use Comets to perform an uncertainty analysis with the Brewery model. This type of experimentation aims at studying and quantifying how the uncertainty on inputs of a model propagates through the model and leads to variability in the outputs of this model.

## The Brewery model

Like in the previous tutorial we are working with the Brewery model. It is a drink consumption model including stock level management and customer satisfaction dynamics. The model input parameters that we will deal with are: 

- NBWaiters: the number of waiters in the bar
- RestockQty: a fix quantity of drinks added to the current stock of the bar once it reaches a treshold value.

The Brewery model has no default outputs as the value of all attributes at the end of a simulation can be considered as an output. In our case we will use the following attributes:

 - AverageSatisfaction: the average customer satisfaction
 - Stock: the stock of the bar.


For this tutorial we suppose that the model's inputs (i.e NBWaiters and RestockQty) are uncertain. The model remains deterministic. We want to answer the following question: **How are the AverageSatisfaction and the stock affected by the uncertainty on the model's inputs?** 

## The approach
An uncertainty analysis requires to specify precisely what is the task we are experimenting with: what are its inputs and ouputs? In our case the inputs are NBWaiters, RestockQty and the outputs are the AverageSatisfaction and the Stock. Along with that we need to precisely specify the uncertainty on the input variables. In this study, we will model the uncertainty on the inputs with two  uniform discrete distributions. The uncertainty analysis will then return the default statistics for the variability of the AverageSatisfaction and the Stock: mean, standard deviation, standard error of the mean, 95% confidence interval of the estimated mean. These results will be used to answer the question raised in the problem statement.

## 1. Define the task on which to experiment
Comets always works with the same method: it performs analyses on tasks. Before defining any experiment we need to define the task on which it will be performed. A task behaves as a function that takes as input a ParameterSet and outputs another ParameterSet. It has a method evaluate that is in charge of performing the evaluation and returning the outputs.
During the task evaluation, the model is run, but when creating the task we also allow for some additional pre-processing (i.e encoding) of the input parameters or post-processing of the outputs of the model.



In [1]:
# code to display the diagram below
from IPython.display import Image
Image(url= "Comets_ModelTask.png", width=1000, height=1600)

To run the analysis we first need to import comets

In [2]:
import comets 

In order to import the Brewery model we then need to specify the path to the project containing it.

In [3]:
#This example assumes that this notebook is in the folder containing the Brewery project
#and that the model has been compiled with the python wrappers activated.

from pathlib import Path
cwd = Path().resolve()

### Import the simulator
We need to import the simulator in order to allow comets to interact with it. In this example, we work with the Brewery simulator, which we can instantiate using a CosmoInterface as follows:

In [4]:
simulator = comets.CosmoInterface(
    simulator_path = 'BreweryTutorialSimulation',
    project_path = cwd,
)

For more details on how to instantiate the Brewery model, refer to the tutorial "Running simulations with Comets".

### Define the input parameters of the task and the encoding for working with the model
The task should take as input a ParameterSet, which is a set of parameters, stored as a dictionary containing names of parameters and their values. In our case the input parameters are the NBWaiters and the RestockQty, which would take the form: 
{'NBWaiters': NBWaiters, 'RestockQty': RestockQty} 



In order for the model to recognize input parameters, we need to encode them to a ParameterSet where the keys are the datapaths of the corresponding attributes in the model, which for NBWaiters and RestockQty corresponds to: 

- 'Model::{Entity}MyBar::@NbWaiters'
- 'Model::{Entity}MyBar::@RestockQty'

The encoding function takes the ParameterSet:

{'NBWaiters': NBWaiters, 'RestockQty': RestockQty} 

and returns the ParameterSet:

{'Model::{Entity}MyBar::@NbWaiters': NbWaiters, 'Model::{Entity}MyBar::@RestockQty': RestockQty}.

In [5]:
def encoder(parameters):
    return {'Model::{Entity}MyBar::@NbWaiters': parameters['NbWaiters'],
            'Model::{Entity}MyBar::@RestockQty': parameters['RestockQty']}
    

### Define the quantities of interest for the experiment, which are the outputs of the task
The quantities we are interested in are the AverageSatisfaction and the Stock. The task should return a ParameterSet with the values of these quantities. The AverageSatisfaction and the Stock are available in the model via their respective datapath :


- ‘Model::{Entity}MyBar::@AverageSatisfaction’
- ‘Model::{Entity}MyBar::@Stock’.

The quantities of interest can be computed from the outputs of the model by creating a function that takes as input the CosmoInterface and returns a ParameterSet with the values of the quantities of interest.

In [6]:
def QuantityOfInterest(simulator):
    outputs = simulator.get_outputs(['Model::{Entity}MyBar::@AverageSatisfaction', 'Model::{Entity}MyBar::@Stock'])
    AverageSatisfaction = outputs['Model::{Entity}MyBar::@AverageSatisfaction']
    stock = outputs['Model::{Entity}MyBar::@Stock']
    return {'AverageSatisfaction' : AverageSatisfaction, 'Stock' : stock}

### Declaring the Task

Once the inputs and outputs have been defined, the final step is to declare the task with its CosmoInterface, encode and get_outcomes method. Evaluating the task corresponds to encoding the inputs, running a simulation and then computing the quantity of interest with the get_outcomes function.

In [7]:
uncertaintytask = comets.ModelTask(simulator, get_outcomes = QuantityOfInterest, encode = encoder)

## 2. Model the uncertainty on the inputs

Once the task has been defined, we need to choose how to represent the uncertainty on its inputs. This is done by declaring the input parameters distribution in the format described below. The distribution is a list of dictionaries, one for each independent parameter. Each distionary contains the following information:

- **name** (str): the name of the parameter, should correspond to a name of input parameter of the task
- **distribution** (str): the name of the probability distribution for this parameter
- **parameters** (dict): the parameters of the distribution, in the form of a dictionary with named parameters and their values
- **shape** (optional, int): the size of the list if the parameter corresponds to a list of independent and identically distributed(i.i.d.) random variables.

The currently available distributions are:
- **normal**: its parameters are {‘mu’: mean of the distribution, ‘sigma’: standard deviation}
- **exponential**: its parameters are {‘scale’: scale parameter, ‘shift’: location of the lower bound}
- **uniform**: its parameters are {‘lower’: lower threshold of distribution, ‘upper’: upper threshold of distribution}
- **discreteuniform**: its parameters are {‘lower’: lower threshold of distribution. Value will be rounded up to closest integer, ‘upper’: upper threshold of distribution. Value will be rounded down to closest integer.
}

In the Brewery model, the NbWaiters and the RestockQty are represented by integers. We will model them with two uniform discrete distributions. The NbWaiters' distribution will range from 1 to 4 (included) and the RestockQty's distribution will range from 5 to 15 (included).

In [8]:
parameters = [
    {
        "name": 'NbWaiters',
        "distribution": "discreteuniform",
        "parameters": {'lower': 1, 'upper': 4},
    },
    {
        "name": 'RestockQty',
        "distribution": "discreteuniform",
        "parameters": {'lower': 5, 'upper': 15},
    }
]

## 3. Declaring the Experiment
An uncertainty analysis is an experiment that can take the following arguments: 
- **task**: the task on which to perform the analysis
- **distribution**: the distribution of the input parameters
- **n_jobs**: the number of samples that are evaluated in parallel. Defaults to 1 (no parallelization)
- **stop_criteria**: dictionary containing the stop criteria of the experiment. In our case we use ‘max_evaluations’ which corresponds to the number of task evaluation to perform. Other criteria are available, more information on them can be found in Comets' documentation. 


In [9]:
ua = comets.UncertaintyAnalysis(
        task = uncertaintytask,
        distribution = parameters,
        n_jobs = 6,
        stop_criteria = {'max_evaluations' : 1000})

## 4. Running the experiment

In [10]:
ua.run()

## 5. Displaying the results of the Experiment
The results of the analysis are stored in the attribute 'results', it contains default statistics on the quantity of interest (the service level).

In [11]:
ua.results

{'AverageSatisfaction': {'mean': 0.9111776447105788,
  'std': 0.4180255013031645,
  'sem': 0.013212522431041365,
  '95% confidence interval': (0.8852502269747994, 0.9371050624463583)},
 'Stock': {'mean': 14.416167664670658,
  'std': 3.216018118536465,
  'sem': 0.10164861090372151,
  '95% confidence interval': (14.216698864453472, 14.615636464887844)}}

It is also possible to access all the results of evaluations of the task in order to compute other statistics. Below we display the first five evaluation results.

In [12]:
ua.list_of_results[:5]

[{'AverageSatisfaction': 1, 'Stock': 18},
 {'AverageSatisfaction': 2, 'Stock': 16},
 {'AverageSatisfaction': 1, 'Stock': 11},
 {'AverageSatisfaction': 1, 'Stock': 17},
 {'AverageSatisfaction': 1, 'Stock': 12}]